In [1]:
import os
import re
import json
from PIL import Image

from element_type import TYPE_DICT, KEYS_USED

DICT_TYPE = {v: k for k, v in TYPE_DICT.items()}  # converted TYPE_DICT

def read_text_file(file_path):
    content = None
    for encoding in ['cp949', 'utf-16', 'utf-8']:
        try:
            with open(file_path, 'r', encoding=encoding) as f:
                content = f.read()
        except Exception:
            pass

    if content is None:
        raise RuntimeError(f"Failed to read {file_path}")
    return content

In [2]:
key_list = list(TYPE_DICT.keys())
key_list.sort()
class_list_for_yolo = []
for key in key_list:
    class_list_for_yolo.append(TYPE_DICT[key])

In [3]:
duplicated_image_removed_list = './unique_imgs.json'
assert os.path.isfile(duplicated_image_removed_list)
with open(duplicated_image_removed_list, 'r') as f:
    unique_imgs_data = f.read()
unique_imgs_data = json.loads(unique_imgs_data)

In [4]:
dataset_name = 'miniwob_dataset_test'
train_label_path = f'../../datasets/{dataset_name}/commentator_labels/train2024'
valid_label_path = f'../../datasets/{dataset_name}/commentator_labels/valid2024'
os.makedirs(train_label_path, exist_ok=True)
os.makedirs(valid_label_path, exist_ok=True)

In [5]:
def save_data(data_list, save_dir_path, save_name):
    file_full_path = os.path.join(save_dir_path, save_name)
    with open(file_full_path,  'w') as f:
        for label_data in data_list:
            f.write(label_data+'\n')

In [ ]:
key_count_dict_train = {key: 0 for key in class_list_for_yolo}
key_count_dict_valid = {key: 0 for key in class_list_for_yolo}

search_button_count = 0
for task_category_key in unique_imgs_data.keys():
    unique_image_list = [action_json for action_json in unique_imgs_data[task_category_key]]
    for image_fullpath in unique_image_list:
        annotation_fullpath = image_fullpath[:-3]+"json"

        original_image = Image.open(image_fullpath)
        # with Utterance offset set 0
        y_offset = 0

        json_data = read_text_file(annotation_fullpath)
        element_list = json.loads(json_data)

        # check seed for split train/valid
        match = re.match(r'^(.+?)_([0-9]+)__', annotation_fullpath)
        if match:
            task_name = match.group(1)
            seeds_digits = match.group(2)

        save_label_path = train_label_path
        key_count_dict = key_count_dict_train
        if seeds_digits in ['1000']:
            save_label_path = valid_label_path
            key_count_dict = key_count_dict_valid

        save_image_name = f"{os.path.split(task_name)[-1]}_{seeds_digits}_{os.path.split(image_fullpath)[-1]}"
        
        converted_datalist = []
        datalist = []
        for element in element_list:
            if 'type' in element.keys():
                if 'subtype' in element and element['subtype'] == 'search':
                    search_button_count+=1
                    continue                    
                type_val = TYPE_DICT[element["type"]]
                keys_used = KEYS_USED[type_val]

                fix_format_element = {}
                for pk in keys_used:
                    if pk in element.keys():
                        fix_format_element[pk] = element[pk]
                        if pk == 'coords':
                            x1, y1, x2, y2 = fix_format_element[pk]
                            fix_format_element[pk] = [x1, y1 - y_offset, x2, y2 - y_offset]
                    else:
                        if pk == 'text':
                            fix_format_element[pk] = ""
                        elif pk == 'coords':
                            raise Exception(f"data soomething wrong : {element}")
                        elif pk in ['focused','highlighted']:
                            fix_format_element[pk] = "False"
                        else:
                            raise Exception(f"data soomething  pk ?  : {pk}")
                datalist.append(json.dumps(fix_format_element))
                fix_format_element.pop('coords')

                fix_format_element['type'] = TYPE_DICT[fix_format_element['type']]
                key_count_dict[fix_format_element['type']] += 1
                converted_datalist.append(json.dumps(fix_format_element))
            else:
                raise Exception(f"Element info invalid : {element}")
        if len(converted_datalist) == 0:
            print(f"Check data : {annotation_fullpath}")
        else:            
            save_label_name = f"{os.path.splitext(save_image_name)[0]}.txt"
            save_data(converted_datalist, save_label_path, save_label_name)
            save_label_name = f"{os.path.splitext(save_image_name)[0]}_annotation.txt" # original
            save_data(datalist, save_label_path, save_label_name)

In [ ]:
print(f"nc: {len(class_list_for_yolo)}")
print(f"names: {class_list_for_yolo}")